In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import csv
import os
import sys
sys.path.append("..")
import minimise
# Enable or disable Tensor Float 32 Execution
tf.config.experimental.enable_tensor_float_32_execution(False)



def write_profile(filename, centers, densities_O, densities_H):
    """
    Write the density profiles to a file.

    Parameters:
    - filename (str): Output file name.
    - centers (np.ndarray): Bin centers.
    - densities_O (np.ndarray): Density values for component O.
    - densities_H (np.ndarray): Density values for component H.
    """
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=' ')
        writer.writerow(["xbins", "rho_O", "rho_H"])
        for center, density_O, density_H in zip(centers, densities_O, densities_H):
            writer.writerow([f"{center:.4f}", f"{density_O:.20f}", f"{density_H:.20f}"])


def do_all_test_SR(modelH_path, modelO_path, data_path, results_path):
    """
    Determine the self-consistent density profiles with neural DFT for all test systems.

    Parameters:
    - model_path (str): Path to the Keras model.
    - data_path (str): Path to the simulation data.
    - results_path (str): Path to save the results.
    """
    model_H = keras.models.load_model(modelH_path)
    model_O = keras.models.load_model(modelO_path)
    sim_data = np.load(data_path, allow_pickle=True).item()
    
    os.makedirs(results_path, exist_ok=True)

    for key, profiles in sim_data["test"].items():
        output_file = f'{results_path}/{key}.out'

        print(f"Processing {key}")
        
        zbins = profiles["xbins"]
        muloc_H = profiles["muloc_H"]
        muloc_O = profiles["muloc_O"]
        
        zs, rho_H, rho_O = minimise.minimise_SR_twotype(model_H, model_O, zbins, muloc_H, muloc_O, initial_guess=0.04, input_bins=667, plot=True, tolerance=1e-6, maxiter=50000)
        if zs is not None:
            write_profile(output_file, zs, rho_H, rho_O)


def do_all_test_LR(modelH_path, modelO_path, data_path, results_path,
                   q_H=1.0, q_O=-1.0, sigma=5.0, temp=4000.0, dielectric=1.0):
    """
    Determine the self-consistent density profiles with neural DFT for all test systems.

    Parameters:
    - model_path (str): Path to the Keras model.
    - data_path (str): Path to the simulation data.
    - results_path (str): Path to save the results.
    """
    model_H = keras.models.load_model(modelH_path)
    model_O = keras.models.load_model(modelO_path)
    sim_data = np.load(data_path, allow_pickle=True).item()
    
    os.makedirs(results_path, exist_ok=True)

    for key, profiles in sim_data["test"].items():
        output_file = f'{results_path}/{key}.out'

        print(f"Processing {key}")
        
        zbins = profiles["xbins"]
        muloc_H = profiles["muloc_H"]
        muloc_O = profiles["muloc_O"]
        
        zs, rho_H, rho_O = minimise.minimise_LR_twotype(model_H, model_O, zbins, muloc_H, muloc_O,
                                                        q_H, q_O, sigma, temp, dielectric, 
                                                        initial_guess=0.01, input_bins=667, plot=True, 
                                                        tolerance=1e-6, maxiter=50000)
        if zs is not None:
            write_profile(output_file, zs, rho_H, rho_O)




In [ ]:

modelH_path = "../../models/RPM_all_H.keras"
modelO_path = "../../models/RPM_all_O.keras"
data_path = "../../data/RPM_all_LR.npy"
results_path = "../../tests/RPM_all_LR"


#do_all_test_SR(modelH_path, modelO_path, data_path, results_path)
#do_all_test_LR(modelH_path, modelO_path, data_path, results_path)
